
# Shopify Orders — Reusable Cleaning Notebook (`shopify_orders.ipynb`)

**Purpose:** Clean and standardize Shopify `orders.csv` for downstream analytics (BigQuery / SQL models / BI dashboards).  
**Inputs:** Raw `orders.csv` exported from Shopify Admin.  
**Outputs:** `orders_clean.csv` (tidy, analysis-ready) + optional BigQuery schema JSON.

---

## How to use
1. Set parameters in the **Parameters** cell below (paths, columns to keep, type overrides).
2. Run notebook top-to-bottom.
3. Upload `orders_clean.csv` to BigQuery (recommended dataset: `shopify_clean.orders`).

**Key features**
- Robust column normalization (snake_case)
- Drop fully empty columns/rows
- Safe type coercion (dates, numerics)
- Handles oddities (leading `'` in ZIPs, mixed currency formats)
- Selects a recommended subset of columns for analytics
- Emits a BigQuery schema JSON (optional convenience)



In [15]:

# ======= Parameters (edit these) =======
INPUT_PATH = "../../data/raw/orders_export_1.csv"              # path to your raw Shopify orders CSV
OUTPUT_PATH = "../../data/clean/orders_clean.csv"         # cleaned CSV output
BIGQUERY_SCHEMA_JSON = "orders_clean_bq_schema.json"  # optional schema file

# Columns to keep (subset). Any missing columns will be ignored gracefully.
KEEP_COLS = [
    "Name", "Email",
    "Financial Status", "Paid at",
    "Fulfillment Status", "Fulfilled at", "Cancelled at",
    "Currency", "Subtotal", "Shipping", "Taxes", "Total", "Discount Amount",
    "Created at",
    "Lineitem name", "Lineitem quantity", "Lineitem price", "Lineitem sku",
    "Vendor",
    "Accepts Marketing", "Source", "Payment Method",
    "Billing Country", "Shipping Country", "Billing Province", "Shipping Province",
    "Risk Level", "Tags"
]

# Date/time columns to coerce -> UTC (if timezone offset present, we keep as tz-aware; BigQuery often expects TIMESTAMP)
DATE_COLS = ["Created at", "Paid at", "Fulfilled at", "Cancelled at"]

# Numeric columns to coerce
NUM_COLS = ["Subtotal", "Shipping", "Taxes", "Total", "Discount Amount", "Lineitem price", "Lineitem quantity"]

# Optional: force certain dtype (after cleaning column names). Keys should be final snake_case names.
# Example: DTYPE_OVERRIDES = {"lineitem_quantity": "Int64", "subtotal": "Float64"}
DTYPE_OVERRIDES = {}


In [16]:

import pandas as pd
import numpy as np
import re
import json 
from pathlib import Path

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 160)


In [17]:

def to_snake(s: str) -> str:
    """Normalize a column name to snake_case and strip unsafe chars."""
    s = s.strip()
    s = re.sub(r"[^0-9a-zA-Z_]+", "_", s)
    s = re.sub(r"__+", "_", s)
    return s.lower().strip("_")

def load_shopify_csv(path: str) -> pd.DataFrame:
    """Load CSV using sensible defaults for Shopify exports."""
    # utf-8-sig handles BOM if present
    df = pd.read_csv(path, dtype=str, encoding="utf-8-sig", keep_default_na=True)
    return df

def strip_apostrophes(series: pd.Series) -> pd.Series:
    """Remove leading apostrophes often seen in ZIP codes like `'53188`."""
    return series.str.replace(r"^'", "", regex=True)

def coerce_numeric(series: pd.Series) -> pd.Series:
    """Coerce a text series to float, handling commas and currency symbols."""
    if series.dtype.name.startswith("float") or series.dtype.name.startswith("int"):
        return series
    cleaned = series.astype(str).str.replace(",", "", regex=False)
    cleaned = cleaned.str.replace(r"[^0-9.\-]", "", regex=True)
    return pd.to_numeric(cleaned, errors="coerce")

def coerce_datetime(series: pd.Series) -> pd.Series:
    """Coerce to pandas datetime. Keeps timezone if provided; otherwise naive."""
    return pd.to_datetime(series, errors="coerce", utc=False)

def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize column names to snake_case."""
    df = df.copy()
    df.columns = [to_snake(c) for c in df.columns]
    return df

def drop_empty(df: pd.DataFrame) -> pd.DataFrame:
    """Drop fully empty rows and columns."""
    df = df.dropna(how="all")
    df = df.dropna(axis=1, how="all")
    return df

def select_columns(df: pd.DataFrame, keep_cols: list) -> pd.DataFrame:
    """Select a subset of columns, ignoring those not present."""
    existing = [c for c in keep_cols if c in df.columns]
    return df[existing].copy()

def summarize_nulls(df: pd.DataFrame) -> pd.DataFrame:
    nulls = df.isna().sum().sort_values(ascending=False).to_frame("null_count")
    nulls["pct_null"] = (nulls["null_count"] / len(df)).round(4)
    return nulls

def bigquery_type_for_series(s: pd.Series) -> str:
    """Infer a reasonable BigQuery type from a pandas Series."""
    if pd.api.types.is_integer_dtype(s):
        return "INTEGER"
    if pd.api.types.is_float_dtype(s):
        return "FLOAT"
    if pd.api.types.is_bool_dtype(s):
        return "BOOL"
    if pd.api.types.is_datetime64_any_dtype(s):
        return "TIMESTAMP"
    return "STRING"

def emit_bq_schema(df: pd.DataFrame, path: str):
    schema = []
    for col in df.columns:
        schema.append({
            "name": col,
            "type": bigquery_type_for_series(df[col]),
            "mode": "NULLABLE"
        })
    Path(path).write_text(json.dumps(schema, indent=2), encoding="utf-8")
    print(f"Wrote BigQuery schema to: {path}")


In [18]:

raw_path = Path(INPUT_PATH)
assert raw_path.exists(), f"Input file not found: {raw_path}"
df_raw = load_shopify_csv(str(raw_path))
print("Loaded rows:", len(df_raw))
display(df_raw.head(3))


Loaded rows: 497


,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References
0,#3317,dduhig624@gmail.com,paid,2025-09-27 20:40:45 +0800,fulfilled,2025-09-30 13:26:31 +0800,no,USD,83.51,0.00,0.00,83.51,NaN,4.39,Free Shipping,2025-09-27 20:40:42 +0800,2,Pumpkin Ceramic Bowl with Lid – Bowl Set for F...,43.95,55.95,NaN,true,true,fulfilled,Diane Duhig,503 Century Oak Drive,503 Century Oak Drive,NaN,NaN,Waukesha,'53188,WI,US,+14142937205,Diane Duhig,503 Century Oak Drive,503 Century Oak Drive,NaN,NaN,Waukesha,'53188,WI,US,+14142937205,NaN,NaN,NaN,PayPal Express Checkout,rQoFzhLAjsLiWgtT3OnIzXffq,0.00,Myves | Home Appliances,6570639458533,NaN,Low,web,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wisconsin,Wisconsin,rQoFzhLAjsLiWgtT3OnIzXffq,NaN,NaN,rQoFzhLAjsLiWgtT3OnIzXffq + aVGKV1HPeQXW8DAxSJ...
1,#3316,rbbchan@hotmail.com,refunded,2025-09-27 05:13:05 +0800,unfulfilled,NaN,no,USD,0.27,0.00,0.00,0.27,7Z1N4P00PHVP,26.68,Free Shipping,2025-09-27 05:13:02 +0800,1,3-in-1 Foldable Cutting Board and Wash Basin -...,26.95,38.95,NaN,true,true,pending,test test,231 Pelican Place,231 Pelican Place,NaN,NaN,Point Roberts,'98281,WA,US,+16723386742,test test,231 Pelican Place,231 Pelican Place,NaN,NaN,Point Roberts,'98281,WA,US,+16723386742,NaN,NaN,NaN,Shopify Payments + PayPal Express Checkout,r7JG6BeN42flJ1DrrsKpeTlxm,0.27,Myves | Home Appliances,6569708683493,NaN,Low,web,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington,Washington,r7JG6BeN42flJ1DrrsKpeTlxm,NaN,NaN,rVcZuGfQw38kZvSmvrZVaMuCN + r7JG6BeN42flJ1Drrs...
2,#3315,estelahooper@icloud.com,paid,2025-09-26 11:58:09 +0800,fulfilled,2025-09-28 03:10:49 +0800,no,USD,149.43,0.00,0.00,149.43,NaN,26.37,Free Shipping,2025-09-26 11:58:09 +0800,2,Pumpkin Ceramic Bowl with Lid – Bowl Set for F...,43.95,55.95,NaN,true,true,fulfilled,Estela Hooper,43 Hawali Cv,43 Hawali Cv,NaN,NaN,Odessa,'79762,TX,US,+14323522541,Estela Hooper,43 Hawaii Cove,43 Hawaii Cove,NaN,NaN,Odessa,'79762,TX,US,+14323522541,NaN,NaN,NaN,Shopify Payments,rcpzJ5SSSVeuDmzPZ74pbqPRQ,0.00,Myves | Home Appliances,6568459993317,NaN,Low,web,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Texas,Texas,rcpzJ5SSSVeuDmzPZ74pbqPRQ,NaN,NaN,rcpzJ5SSSVeuDmzPZ74pbqPRQ


In [19]:

df = df_raw.copy()
df = drop_empty(df)
# Select a subset first (based on original headers), then normalize names
subset_cols = [c for c in KEEP_COLS if c in df.columns]
df = df[subset_cols].copy()
df.columns = [to_snake(c) for c in df.columns]

print("Columns after selection & normalization:", list(df.columns))
display(df.head(3))


Columns after selection & normalization: ['name', 'email', 'financial_status', 'paid_at', 'fulfillment_status', 'fulfilled_at', 'cancelled_at', 'currency', 'subtotal', 'shipping', 'taxes', 'total', 'discount_amount', 'created_at', 'lineitem_name', 'lineitem_quantity', 'lineitem_price', 'lineitem_sku', 'vendor', 'accepts_marketing', 'source', 'payment_method', 'billing_country', 'shipping_country', 'billing_province', 'shipping_province', 'risk_level']


,name,email,financial_status,paid_at,fulfillment_status,fulfilled_at,cancelled_at,currency,subtotal,shipping,taxes,total,discount_amount,created_at,lineitem_name,lineitem_quantity,lineitem_price,lineitem_sku,vendor,accepts_marketing,source,payment_method,billing_country,shipping_country,billing_province,shipping_province,risk_level
0,#3317,dduhig624@gmail.com,paid,2025-09-27 20:40:45 +0800,fulfilled,2025-09-30 13:26:31 +0800,NaN,USD,83.51,0.00,0.00,83.51,4.39,2025-09-27 20:40:42 +0800,Pumpkin Ceramic Bowl with Lid – Bowl Set for F...,2,43.95,NaN,Myves | Home Appliances,no,web,PayPal Express Checkout,US,US,WI,WI,Low
1,#3316,rbbchan@hotmail.com,refunded,2025-09-27 05:13:05 +0800,unfulfilled,NaN,NaN,USD,0.27,0.00,0.00,0.27,26.68,2025-09-27 05:13:02 +0800,3-in-1 Foldable Cutting Board and Wash Basin -...,1,26.95,NaN,Myves | Home Appliances,no,web,Shopify Payments + PayPal Express Checkout,US,US,WA,WA,Low
2,#3315,estelahooper@icloud.com,paid,2025-09-26 11:58:09 +0800,fulfilled,2025-09-28 03:10:49 +0800,NaN,USD,149.43,0.00,0.00,149.43,26.37,2025-09-26 11:58:09 +0800,Pumpkin Ceramic Bowl with Lid – Bowl Set for F...,2,43.95,NaN,Myves | Home Appliances,no,web,Shopify Payments,US,US,TX,TX,Low


In [20]:

# Fix common odd ZIP apostrophes if present
for col in ["billing_zip", "shipping_zip"]:
    if col in df.columns:
        df[col] = strip_apostrophes(df[col].astype(str))

# Datetime coercion
for col in [to_snake(c) for c in DATE_COLS if c in df_raw.columns]:
    if col in df.columns:
        df[col] = coerce_datetime(df[col])

# Numeric coercion
for col in [to_snake(c) for c in NUM_COLS if c in df_raw.columns]:
    if col in df.columns:
        df[col] = coerce_numeric(df[col])

# Optional dtype overrides
for col, dtype in (DTYPE_OVERRIDES or {}).items():
    if col in df.columns:
        try:
            df[col] = df[col].astype(dtype)
        except Exception as e:
            print(f"Warning: could not apply dtype {dtype} to column {col}: {e}")


In [21]:

# Remove fully empty rows again (after coercion)
df = drop_empty(df)

# Shopify order id is usually in 'name' like '#1234'
if 'name' in df.columns:
    dup_count = df.duplicated(subset=['name', 'lineitem_name'], keep='first').sum()
    if dup_count > 0:
        print(f"Found {dup_count} duplicated (name, lineitem_name) rows -> dropping duplicates.")
        df = df.drop_duplicates(subset=['name', 'lineitem_name'], keep='first')

# Basic quality summary
print("Rows after cleaning:", len(df))
display(df.head(5))

print("\nNull summary (top 20):")
display(summarize_nulls(df).head(20))

# Sanity checks (soft asserts as warnings)
def warn_if(cond, msg):
    if cond:
        print("WARNING:", msg)

warn_if('name' not in df.columns, "Missing 'name' (order id) column.")
warn_if('created_at' not in df.columns, "Missing 'created_at' column.")
warn_if('total' in df.columns and df['total'].isna().mean() > 0.2, "More than 20% of 'total' is null.")


Rows after cleaning: 497


,name,email,financial_status,paid_at,fulfillment_status,fulfilled_at,cancelled_at,currency,subtotal,shipping,taxes,total,discount_amount,created_at,lineitem_name,lineitem_quantity,lineitem_price,lineitem_sku,vendor,accepts_marketing,source,payment_method,billing_country,shipping_country,billing_province,shipping_province,risk_level
0,#3317,dduhig624@gmail.com,paid,2025-09-27 20:40:45+08:00,fulfilled,2025-09-30 13:26:31+08:00,NaT,USD,83.51,0.0,0.0,83.51,4.39,2025-09-27 20:40:42+08:00,Pumpkin Ceramic Bowl with Lid – Bowl Set for F...,2,43.95,NaN,Myves | Home Appliances,no,web,PayPal Express Checkout,US,US,WI,WI,Low
1,#3316,rbbchan@hotmail.com,refunded,2025-09-27 05:13:05+08:00,unfulfilled,NaT,NaT,USD,0.27,0.0,0.0,0.27,26.68,2025-09-27 05:13:02+08:00,3-in-1 Foldable Cutting Board and Wash Basin -...,1,26.95,NaN,Myves | Home Appliances,no,web,Shopify Payments + PayPal Express Checkout,US,US,WA,WA,Low
2,#3315,estelahooper@icloud.com,paid,2025-09-26 11:58:09+08:00,fulfilled,2025-09-28 03:10:49+08:00,NaT,USD,149.43,0.0,0.0,149.43,26.37,2025-09-26 11:58:09+08:00,Pumpkin Ceramic Bowl with Lid – Bowl Set for F...,2,43.95,NaN,Myves | Home Appliances,no,web,Shopify Payments,US,US,TX,TX,Low
3,#3315,estelahooper@icloud.com,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2025-09-26 11:58:09+08:00,Pumpkin Ceramic Bowl with Lid – Bowl Set for F...,2,43.95,NaN,Myves | Home Appliances,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,#3314,dr.gendron75@gmail.com,paid,2025-09-24 06:26:50+08:00,fulfilled,2025-09-26 16:06:33+08:00,NaT,USD,53.95,0.0,0.0,53.95,0.00,2025-09-24 06:26:50+08:00,Stainless Steel BBQ Tools Set -18 Pcs Grilling...,1,53.95,NaN,Myves | Home Appliances,no,web,Shopify Payments,US,US,CT,CT,Low



Null summary (top 20):


,null_count,pct_null
cancelled_at,491,0.9879
lineitem_sku,138,0.2777
fulfilled_at,40,0.0805
payment_method,32,0.0644
paid_at,32,0.0644
discount_amount,26,0.0523
source,26,0.0523
accepts_marketing,26,0.0523
shipping_country,26,0.0523
billing_province,26,0.0523


In [22]:
# Define output paths relative to notebook
out_path = Path("../../data/clean/orders_clean.csv")
schema_path = Path("../../data/clean/orders_clean_bq_schema.json")

# Save cleaned CSV
df.to_csv(out_path, index=False, encoding="utf-8")
print(f"✅ Saved cleaned CSV → {out_path.resolve()}  (rows={len(df)})")

# Emit a convenience BigQuery schema JSON (optional)
emit_bq_schema(df, schema_path)
print(f"✅ Wrote BigQuery schema JSON → {schema_path.resolve()}")



✅ Saved cleaned CSV → /Users/alvychen/Desktop/Ecommerce_Growth_Analytics/data/clean/orders_clean.csv  (rows=497)
Wrote BigQuery schema to: ../../data/clean/orders_clean_bq_schema.json
✅ Wrote BigQuery schema JSON → /Users/alvychen/Desktop/Ecommerce_Growth_Analytics/data/clean/orders_clean_bq_schema.json


In [23]:

# Optional: quick preview of daily revenue (computed naively at line-level).
# NOTE: In BigQuery you'll create a proper order-level view to avoid double-counting.
if 'created_at' in df.columns and 'total' in df.columns:
    tmp = (df[['created_at','total']]
           .dropna(subset=['created_at'])
           .assign(order_date=lambda x: x['created_at'].dt.floor('D'))
           .groupby('order_date', as_index=False)['total'].sum())
    display(tmp.tail(10))
else:
    print("Skipping metrics preview: missing 'created_at' or 'total'.")


,order_date,total
228,2025-09-12 00:00:00+08:00,39.75
229,2025-09-14 00:00:00+08:00,53.95
230,2025-09-16 00:00:00+08:00,130.90
231,2025-09-17 00:00:00+08:00,31.80
232,2025-09-21 00:00:00+08:00,39.75
233,2025-09-22 00:00:00+08:00,72.77
234,2025-09-23 00:00:00+08:00,39.75
235,2025-09-24 00:00:00+08:00,53.95
236,2025-09-26 00:00:00+08:00,149.43
237,2025-09-27 00:00:00+08:00,83.78
